In [40]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
from imblearn.over_sampling import SMOTE
from functools import partial
from collections import OrderedDict
import warnings
warnings.filterwarnings("ignore")


In [41]:
def maxmin1(x):
 
    max_value = x.max().item()
    print ('1:', max_value)
    min_value = x.min().item()  
    print ('2:', min_value)
    x_normalized = (x - min_value) / (max_value - min_value)
     
    return x_normalized

In [42]:
def partition(RG_GRFf_file_path, RG_GRFl_file_path, abn_ratio):

    with open(RG_GRFf_file_path, 'rb') as file:
        GRFf = pickle.load(file).astype(np.float32)
    file.close()

    with open(RG_GRFl_file_path, 'rb') as file:
        GRFl = pickle.load(file).astype(np.float32)
    file.close()
    
    # stat indice for 0(abnormal)/1(healthy)
    GRFl_0_indice = np.where(GRFl==0)
    GRFl_1_indice = np.where(GRFl==1)
    
    # print ('6:', GRFl_0_indice)
    # print ('7:', GRFl_1_indice)
    
    # select the corresponding features for indice
    GRFl_features_0 = GRFf[list(GRFl_0_indice[0])]
    GRFl_features_1 = GRFf[list(GRFl_1_indice[0])]
    
    # select the corresponding features for indice
    GRFl_0 = GRFl[list(GRFl_0_indice[0])]
    GRFl_1 = GRFl[list(GRFl_1_indice[0])]

    # print (GRFl_features_0.shape)
    
    # print (GRFl_features_1.shape)
    
    # 67977 abnoraml / 16574 healthy, 因此至少
    # print (round(len(GRFl_0)/len(GRFl_1),3))
    
    # print (GRFl_0.shape)
    
    # print (GRFl_1.shape)

    # 训练样本
    #### Portation of  0(abnormal)
    total_sample_size = GRFl_features_0.shape[0]
    total_sample_range = list(np.arange(0, total_sample_size))
    train_sample_size = int(GRFl_features_0.shape[0] * abn_ratio)
    # 训练
    train_indices = np.random.choice(GRFl_features_0.shape[0], size = train_sample_size, replace=False)
    # print ('1:', indices)
    # print ('2:', type(indices))
    # print ('3:', indices.shape)
    GRFl_features_0_portion_train = GRFl_features_0[train_indices]
    GRFl_0_portion_train = GRFl_0[train_indices]
    # print ('4:', GRFl_features_0_portion)
    # print ('5:', type(GRFl_features_0_portion))
    # print ('6:', GRFl_features_0_portion.shape)
    # print ('44:', GRFl_0_portion)
    # print ('7:', type(GRFl_0_portion))
    # print ('8:', GRFl_0_portion.shape)    
    #### Concatenate of  0(abnormal) + 1(healthy)
    ## 3398 + 16574 = 19972
    GRFf_train = np.concatenate((GRFl_features_0_portion_train, GRFl_features_1),axis = 0)
    GRFl_train = np.concatenate((GRFl_0_portion_train, GRFl_1),axis = 0)
    # print ('9:', type(GRFf)) 
    # print ('10:', GRFf.shape) 
    # print ('11:', type(GRFl)) 
    # print ('12:', GRFl.shape)
    
    # 测试样本
    test_indices = []
    for e in total_sample_range:
        if e not in train_indices:
            # test_indices.append(np.where(total_sample_range==e)[0][0])
            test_indices.append(e)
            
    ##
    test_indices = np.array(test_indices)
    GRFl_features_0_portion_test = GRFl_features_0[test_indices]
    GRFl_0_portion_test = GRFl_0[test_indices]
    ##
    GRFf_test = np.concatenate((GRFl_features_0_portion_test, GRFl_features_1), axis = 0)
    GRFl_test = np.concatenate((GRFl_0_portion_test, GRFl_1), axis = 0)

    ####
    print ('13:', GRFf_train.shape)
    print ('14:', GRFl_train.shape)
    print ('15:', GRFf_test.shape)
    print ('16:', GRFl_test.shape)

    return GRFf_train, GRFl_train, GRFf_test, GRFl_test

In [43]:
def balance_01(X, Y):
    GRFf = X
    GRFl = Y
    # 使用SMOTE进行过采样时正样本和负样本要放在一起，生成比例1：1
    smo = SMOTE(n_jobs=-1)
    # 这里必须是fit_resample()，有些版本是fit_sample()无法运行
    #### reshape, 用KNN来插值维度必须是1D
    GRFf_1d = GRFf.reshape(GRFf.shape[0], -1)
    # print ('test1:', GRFf.shape)
    # 特征维度
    GRFf_1d_fnum = GRFf.shape[1]
    
    # print ('1:', GRFf_1d.shape)
    GRFf_re, GRFl_re = smo.fit_resample(GRFf_1d, GRFl)
    
    # print ('2:', GRFf_re.shape)
    # print ('3:', GRFl_re.shape)
    # 样本
    GRFf_re = GRFf_re.reshape(GRFf_re.shape[0], GRFf_1d_fnum, -1)
    #
    # print ('4:', GRFf_re.shape)
    
    
    #### test the distribution proportion
    #
    #
    GRFl_re_0_indice = np.where(GRFl_re==0)
    GRFl_re_1_indice = np.where(GRFl_re==1)
    
    #### 1:1，少数类样本0(疾病人群)数量增加为16574(0.05)
    # print ('5:', len(list(GRFl_re_0_indice[0]))) 
    # print ('6:', len(list(GRFl_re_1_indice[0]))) 
    # print ('7:', GRFf_re)
    #
    #
    #
    print ('17:', len(list(GRFl_re_0_indice[0])))
    print ('18:', len(list(GRFl_re_1_indice[0])))    
    
    return GRFf_re, GRFl_re

In [ ]:
#### DataSet 

In [44]:
# ---------------1、load---------------
RG_GRFf_file_path = 'DataSet1/GRFf.pkl'
RG_GRFl_file_path = 'DataSet1/GRFl.pkl'

# part ratio，这里健康样本是陪衬，一直要加上
abn_ratio = 0.3 # 30%训练
# abn_ratio = 0.2 # 20%训练
# abn_ratio = 0.1 # 10%
# abn_ratio = 0.05 # 5%


# 按比例切分样本，但目前是固定10%:90%, 20%:80%, 30%:70%, 但目前的10% 20% 30%必须变起来写成随机化的(sampler???)
GRFf_train_part, GRFl_train_part, GRFf_val_part, GRFl_val_part  =  partition(RG_GRFf_file_path, RG_GRFl_file_path, abn_ratio)
# 平衡训练集样本量，正负样本数为1:1
GRFf_train_ba, GRFl_train_ba = balance_01(GRFf_train_part, GRFl_train_part)
# 平衡训练集样本量，正负样本数为1:1
GRFf_val_ba, GRFl_val_ba = balance_01(GRFf_val_part, GRFl_val_part)

# ------------------------ 2、normlization max-min scaler------------------------
GRFf_train = torch.from_numpy(GRFf_train_ba)
GRFf_train = maxmin1(GRFf_train)

GRFf_val = torch.from_numpy(GRFf_val_ba)
GRFf_val = maxmin1(GRFf_val)

#
GRFl_train = torch.from_numpy(GRFl_train_ba)
GRFl_val = torch.from_numpy(GRFl_val_ba)

# ------------------------ 3、Tensor Dataset------------------------
train_set = TensorDataset(GRFf_train, GRFl_train)
val_set = TensorDataset(GRFf_val, GRFl_val)



13: (36967, 10, 101)
14: (36967,)
15: (64158, 10, 101)
16: (64158,)
17: 20393
18: 20393
17: 47584
18: 47584
1: 1.6962571144104004
2: -0.43713119626045227
1: 1.6862719058990479
2: -0.43713119626045227


In [45]:
# -----------------4、dataloader------------------------
abn_ratio = 0.01 # 1% 
val_percent = 0.3
# train_batch_size = 32
train_batch_size = 128

# test_batch_size = 48
test_batch_size = 32

workers  = 3
pin_memory = True

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size = train_batch_size,
    shuffle = True,
    num_workers = workers,
    pin_memory = pin_memory,
    sampler = None
)
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size = test_batch_size,
    shuffle = False,
    num_workers = workers,
    pin_memory = pin_memory
)

print ('11:', train_loader)
print ('12:', val_loader)

print ('13:', len(train_loader))
print ('14:', len(val_loader))

11: <torch.utils.data.dataloader.DataLoader object at 0x00000222771E8450>
12: <torch.utils.data.dataloader.DataLoader object at 0x000002218E59F450>
13: 319
14: 2974


In [ ]:
### ViT Model

In [51]:
# 1、 Linear Projection of Flattened Patches

class PatchEmbed(nn.Module):
    """
    2D Image to Patch Embedding
    """
    # 输入图片大小，patch大小，输入图片channel，token大小（16×16×3）
    def __init__(self, img_size=224, patch_size=16, in_c=3, embed_dim=768, norm_layer=None):
        super().__init__()
        img_size = (img_size, img_size)
        patch_size = (patch_size, patch_size)
        self.img_size = img_size
        # self.patch_size = patch_size
        # self.patch_size = 3
        self.grid_size = (img_size[0] // patch_size[0], img_size[1] // patch_size[1])
        # self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.num_patches = 101

 # RuntimeError: Expected 4-dimensional input for 4-dimensional weight [768, 10, 16, 16], but got 3-dimensional input of size [32, 10, 101] instead

        self.proj = nn.Conv1d(in_c, embed_dim, kernel_size=3, stride=1, padding=1)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()
 
    def forward(self, x):
        # B, C, H, W = x.shape
        # 如果传入图片的高和宽和预设不同会报错
        # assert H == self.img_size[0] and W == self.img_size[1], \
        #     f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        B,C,L = x.shape 
        # flatten: [B, C, H, W] -> [B, C, HW]
        # transpose: [B, C, HW] -> [B, HW, C]
        # # 128 × 768 × 101 → 128 × 101 × 768(x)
        x = self.proj(x).transpose(1, 2)
        x = self.norm(x)
        return x


# 2、 Attention Module
class Attention(nn.Module):
    def __init__(self,
                 dim,  # 输入token的dim
                 num_heads=8,  # 几头
                 qkv_bias=False,
                 qk_scale=None,
                 attn_drop_ratio=0.,
                 proj_drop_ratio=0.):
        super(Attention, self).__init__()
        self.num_heads = num_heads
        # 得到q，k，v之后，根据num_heads对q，k，v进行切块，切块后的dim就是dim/头数
        head_dim = dim // num_heads
        # 就是根号d，等于q，k的维度
        self.scale = qk_scale or head_dim ** -0.5
        # 直接通过一个全连接层生成q，k，v
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop_ratio)
        # 表示映射矩阵Wo
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop_ratio)
 
    def forward(self, x):
        # [batch_size, num_patches + 1, total_embed_dim]
        B, N, C = x.shape
 
        # qkv(): -> [batch_size, num_patches + 1, 3 * total_embed_dim]
        # reshape: -> [batch_size, num_patches + 1, 3, num_heads, embed_dim_per_head]
        # permute: -> [3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)
 
        # transpose: -> [batch_size, num_heads, embed_dim_per_head, num_patches + 1]
        # @: multiply -> [batch_size, num_heads, num_patches + 1, num_patches + 1]
        # @是矩阵乘法，*scale就是是乘根号d
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
 
        # @: multiply -> [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        # transpose: -> [batch_size, num_patches + 1, num_heads, embed_dim_per_head]
        # reshape: -> [batch_size, num_patches + 1, total_embed_dim]
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        # *Wo
        x = self.proj(x)
        x = self.proj_drop(x)
        return x



# 3、 MLP Block
class Mlp(nn.Module):
    """
    MLP as used in Vision Transformer, MLP-Mixer and related networks
    """
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
 
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x



# 4、 Transformer Block
class Block(nn.Module):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,  # 第一个Linear输出倍数
                 qkv_bias=False,
                 qk_scale=None,
                 drop_ratio=0.,
                 attn_drop_ratio=0.,
                 drop_path_ratio=0.,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm):
        super(Block, self).__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                              attn_drop_ratio=attn_drop_ratio, proj_drop_ratio=drop_ratio)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path_ratio) if drop_path_ratio > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop_ratio)
 
    def forward(self, x):
        # Res1
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x




# 5、 Vision Transformer network
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_c=10, num_classes=1,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0, qkv_bias=True,
                 qk_scale=None, representation_size=None, distilled=False, drop_ratio=0.,
                 attn_drop_ratio=0., drop_path_ratio=0., embed_layer=PatchEmbed, norm_layer=None,
                 act_layer=None):
        """
        Args:
            img_size (int, tuple): input image size
            patch_size (int, tuple): patch size
            in_c (int): 输入通道数
            num_classes (int): 多分类数量
            embed_dim (int): embedding后维度
            depth (int): Encoder数量
            num_heads (int): multi-head头数
            mlp_ratio (int): mlp隐藏层维度是输入的多少倍
            qkv_bias (bool): qkv的Linear过程有没有偏置？
            qk_scale (float): 类似根号dimk
            representation_size (Optional[int]): MLP head是否只有一个全连接层，对应Pre-logits，是一个可选项
            distilled (bool): 用于Deit的，与Vit无关
            drop_ratio (float): dropout rate
            attn_drop_ratio (float): attention dropout rate
            drop_path_ratio (float): stochastic depth rate
            embed_layer (nn.Module): Embedding层
            norm_layer: (nn.Module): normalization层
        """
        super(VisionTransformer, self).__init__()
        self.num_classes = num_classes # 1000
        self.num_features = self.embed_dim = embed_dim  # 768
        self.num_tokens = 2 if distilled else 1 # 不管
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer = act_layer or nn.GELU
        # 224,16,3,768
        self.patch_embed = embed_layer(img_size=img_size, patch_size=patch_size, in_c=in_c, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches # 101
 
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) # 1,1,768
        self.dist_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None # 不管
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + self.num_tokens, embed_dim)) # 位置编码，注意要+1（cls token）
        self.pos_drop = nn.Dropout(p=drop_ratio) # 位置编码后的dropout
 
        dpr = [x.item() for x in torch.linspace(0, drop_path_ratio, depth)]  # 构建一个dropout的等差序列，但默认为0
        # *可以解引用
        self.blocks = nn.Sequential(*[
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                  drop_ratio=drop_ratio, attn_drop_ratio=attn_drop_ratio, drop_path_ratio=dpr[i],
                  norm_layer=norm_layer, act_layer=act_layer)
            for i in range(depth)
        ])
        self.norm = norm_layer(embed_dim)
 
        # 之前所说的可选项,略过
        if representation_size and not distilled:
            self.has_logits = True
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ("fc", nn.Linear(embed_dim, representation_size)),
                ("act", nn.Tanh())
            ]))
        else:
            self.has_logits = False
            self.pre_logits = nn.Identity() # 什么也不做
 
        # 分类，不用管else 768 --> 1000
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()
 
        # 以下与Vit无关 //开始
        self.head_dist = None
        if distilled:
            self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if num_classes > 0 else nn.Identity()
        # 结束//
 
        # 权重初始化
        nn.init.trunc_normal_(self.pos_embed, std=0.02) # 位置编码
        if self.dist_token is not None:
            nn.init.trunc_normal_(self.dist_token, std=0.02)
 
        nn.init.trunc_normal_(self.cls_token, std=0.02) # cls token
        self.apply(_init_vit_weights)
 
    def forward_features(self, x):
        # [B, C, H, W] -> [B, num_patches, embed_dim]
        x = self.patch_embed(x)  # [B, 196, 768]
        # [1, 1, 768] -> [B, 1, 768]
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)
        if self.dist_token is None:
            x = torch.cat((cls_token, x), dim=1)  # [B, 197, 768]
        else:
            x = torch.cat((cls_token, self.dist_token.expand(x.shape[0], -1, -1), x), dim=1)
 
        x = self.pos_drop(x + self.pos_embed)
        x = self.blocks(x)
        x = self.norm(x)
        if self.dist_token is None:
            return self.pre_logits(x[:, 0])
        else:
            return x[:, 0], x[:, 1]
 
    def forward(self, x):
        x = self.forward_features(x)
        if self.head_dist is not None:
            x, x_dist = self.head(x[0]), self.head_dist(x[1])
            if self.training and not torch.jit.is_scripting():
                # during inference, return the average of both classifier predictions
                return x, x_dist
            else:
                return (x + x_dist) / 2
        else:
            x = self.head(x)
        return x
 
 
def _init_vit_weights(m):
    """
    ViT weight initialization
    :param m: module
    """
    if isinstance(m, nn.Linear):
        nn.init.trunc_normal_(m.weight, std=.01)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv1d):
        nn.init.kaiming_normal_(m.weight, mode="fan_out")
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LayerNorm):
        nn.init.zeros_(m.bias)
        nn.init.ones_(m.weight)
 
def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output
 
 
class DropPath(nn.Module):
    """
    Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob
 
    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)
    
# def vit_base_patch16_224(num_classes: int = 1000):
#     """
#     ViT-Base model (ViT-B/16) from original paper (https://arxiv.org/abs/2010.11929).
#     ImageNet-1k weights @ 224x224, source https://github.com/google-research/vision_transformer.
#     weights ported from official Google JAX impl:
#     链接: https://pan.baidu.com/s/1zqb08naP0RPqqfSXfkB2EA  密码: eu9f
#     """
#     model = VisionTransformer(img_size=224,
#                               patch_size=16,
#                               embed_dim=768,
#                               depth=12,
#                               num_heads=12,
#                               representation_size=None,
#                               num_classes=num_classes)
#     return model
 
# vit_base_patch16_224()


In [ ]:
### DataSet API

In [ ]:
### Setting

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VisionTransformer(img_size=224,
                          patch_size=16,
                          embed_dim=768,
                          depth=12,
                          num_heads=12,
                          representation_size=None,
                          num_classes=1)
print (net)
net=net.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(net.parameters())
# 0.1 reduce / 10 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)# 0.1 reduce / 10 epochs


###
###
# epochs = 100
epochs = 3

# acc
def accuracy_fn(y_true, y_pred):
    y_pred = y_pred>0.5
    y_pred = torch.where(y_pred, 1, 0)
    correct = torch.eq(y_true, y_pred).sum().item() 
    acc = (correct / len(y_pred)) * 100
    return acc


# confusion matrix
# 混淆矩阵计算
def calculate_performance_metrics(y_true, y_pred, labels):

    # 预测矩阵转换0 1矩阵
    y_pred = y_pred>0.5
    y_pred = torch.where(y_pred,1,0)

    # print ('0:', y_pred)
    # print ('00:', y_true)
    # print ('0_:', y_pred.shape)
    # print ('00_:', y_true.shape)
    

    # 再转换为numpy类型
    # y_pred = y_pred.detach().numpy()
    y_pred = y_pred.cpu().numpy()
    
    # y_true = y_true.detach().numpy()
    y_true = y_true.cpu().numpy()

    # print ('11:', type(y_pred))
    # print ('22:', type(y_true))
    # print ('33:', y_pred.shape)
    # print ('44:', y_true.shape)

    # 计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred)
    # 混淆矩阵画图
#     fx = plot_cm(cm)
    
    # 计算精度
    precision = precision_score(y_true, y_pred, labels = labels, average = 'macro')
    
    # 计算召回率
    recall = recall_score(y_true, y_pred, labels = labels, average = 'macro')
    
    # 计算F1分数
    f1 = f1_score(y_true, y_pred, labels = labels, average = 'macro')
    
    # 计算平衡F1分数
    # 这里假设有一个函数balanced_accuracy_score来计算平衡F1分数
    balanced_f1 = balanced_accuracy_score(y_true, y_pred)
    
    return cm, precision, recall, f1, balanced_f1

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv1d(10, 768, kernel_size=(3,), stride=(1,), padding=(1,))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise

In [ ]:
### Timing

In [55]:
from timeit import default_timer as timer
def print_train_time(start:float,
                     end:float,
                     device:torch.device =None):
    total_time = end - start
    print(f"Train time on {device}:{total_time:.3f} seconds")
    return total_time
 
start_time = timer()
# 模型的运算在此处进行……
end_time = timer()
print_train_time(start=start_time,end=end_time,device="cpu")
 
# 运行结果
# Train time on cpu:0.000 seconds
# 3.819999983534217e-05

Train time on cpu:0.000 seconds


3.100000321865082e-05

In [54]:
### Training

In [57]:
from tqdm import *
torch.manual_seed(42)
train_time_start_on_cpu = timer() 
 
##############################################
for epoch in tqdm(range(epochs)):  
    print(f"Epoch:{epoch}\n------")
    train_loss = 0 
    for batch,(X,y) in enumerate(train_loader): 
        #
        X, y = X.to(device), y.to(device)

        # print ('55:', X.shape)
        # print ('66:', y.shape)
        net.train() 
        y_pred = net(X).squeeze()
        #
        # print ('1:', y_pred)
        # print ('2:', y_pred.shape)
        # print ('3:', y)
        # print ('4:', y.shape)        
 
        loss = criterion(y_pred, y)
        # print ('train_loss:', loss)
        train_loss+=loss
        
        optimizer.zero_grad()
 
        loss.backward()
        # 
        optimizer.step()
        scheduler.step()
 
        # if batch % 400 == 0:
        if batch % 400 == 0:
            print(f"Look at {batch * len(X)}/{len(train_loader.dataset)} samples.")
            # break
 
    train_loss /= len(train_loader)
    

    ###################val/per epoch###########################
    val_loss, val_acc = 0, 0 
    precision_t = 0
    recall_t = 0
    f1_t = 0
    balanced_f1_t = 0
    cm_t = np.zeros((2,2))
    net.eval() 

    with torch.inference_mode():
        for x_val, y_val in val_loader:
            #
            x_val, y_val = x_val.to(device), y_val.to(device)
            val_pred = net(x_val).squeeze() 
            val_loss+=criterion(val_pred, y_val)
            # print ('test_loss:', loss)
            # print ('val_pred:', val_pred)
            # print ('y_val:', y_val)
            # print ('val_pred:', val_pred.shape)
            # print ('y_val:', val_pred.shape)
            val_acc += accuracy_fn(y_true = y_val, y_pred = val_pred)
            cm, precision, recall, f1, balanced_f1 = calculate_performance_metrics(y_true = y_val, y_pred = val_pred, labels=[0,1])
            
            precision_t += precision
            recall_t += recall
            f1_t += f1
            balanced_f1_t += balanced_f1
            cm_t += cm
            
        val_loss /= len(val_loader)
        val_acc /= len(val_loader)
        precision_t /=len(val_loader)
        recall_t /=len(val_loader)
        f1_t /=len(val_loader)
        balanced_f1_t /=len(val_loader)
        
        
    
    print(f"\nTrain loss:{train_loss:.4f} | Test loss:{val_loss:.4f}, Test acc:{val_acc:.4f}")
    
    print ('avg-precision:', precision_t)
    print ('avg-recall:', recall_t)
    print ('avg-f1:', f1_t)
    print ('avg-balanced_f1:', balanced_f1_t)
    print ('last-cm:', cm_t)


# 计算训练时间
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,
                                            end=train_time_end_on_cpu,
                                            device=str(next(net.parameters()).device))

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Epoch:0
------
Look at 0/40786 samples.


 33%|██████████████████████████▋                                                     | 1/3 [33:58<1:07:56, 2038.16s/it]


Train loss:1.9252 | Test loss:1.9271, Test acc:50.0000
avg-precision: 0.25
avg-recall: 0.25
avg-f1: 0.25
avg-balanced_f1: 0.5
last-cm: [[47584. 47584.]
 [95168. 47584.]]
Epoch:1
------
Look at 0/40786 samples.


 33%|██████████████████████████                                                    | 1/3 [1:07:00<2:14:01, 4020.98s/it]


KeyboardInterrupt: 

In [ ]:
### vis testing

In [ ]:
### completed testing